In [2]:
!pip install -q langchain==1.0.5 langchain-community==0.4.1 langchain_google_genai==3.0.1 langchain-core==1.0.4 faiss-cpu==1.12.0 python-dotenv==1.2.1 pypdf==6.2.0 serpapi==0.1.5

In [ ]:
!pip show langchain langchain-community langchain_google_genai langchain-core faiss-cpu python-dotenv pypdf serpapi

Name: langchain
Version: 1.0.5
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 
---
Name: langchain-community
Version: 0.4.1
Summary: Community contributed LangChain integrations.
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: aiohttp, dataclasses-json, httpx-sse, langchain-classic, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
---
Name: langchain-google-genai
Version: 3.0.1
Summary: An integration package connecting Google's genai package and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: filetype, google-ai-generativelanguage, langchain-core, pydantic
Required-by: 
---
Name: langchain-core
Version

In [ ]:
from dotenv import load_dotenv
import os
from langchain_community.utilities import SerpAPIWrapper
from langchain_classic.chains import RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
# from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage,ToolMessage
import json
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

In [ ]:
from google.colab import userdata
import os
# Load Gemini API key from Colab secrets
GOOGLE_API_KEY=userdata.get('GEMINI_API_KEY')
SERPAPI_API_KEY=userdata.get('SERPAPI_API_KEY')

In [ ]:
messages = []

# ✅ Load API key
# load_dotenv()
# SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY") # Removed
if not SERPAPI_API_KEY: # Changed to use the variable loaded from Colab secrets
    raise ValueError("❌ SERPAPI_API_KEY not found. Please check your .env file.")

In [ ]:
# ✅ Load and process PDF
pdf_path = "/content/scholarship_info.pdf"
loader = PyPDFLoader(pdf_path)
pages = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(pages)

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001", google_api_key=GOOGLE_API_KEY)
vectorstore = FAISS.from_documents(documents=docs, embedding=embeddings)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [ ]:
# ✅ Set up LLM and RAG chain
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    google_api_key=GOOGLE_API_KEY
)
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

In [ ]:
# ✅ Define tools using modern `@tool` decorator
@tool
def pdf_scholarship_search(query: str) -> str:
    """Answer scholarship-related questions using the loaded PDF."""
    # Retrieve relevant docs
    return retriever.invoke(query)

@tool
def web_search(query: str) -> str:
    """Search the web using SerpAPI for up-to-date information."""
    try:
        search = SerpAPIWrapper()
        return search.run(query)
    except Exception as e:
        return f"❌ Failed to search the web: {str(e)}"

In [ ]:
# ✅ Bind tools to LLM
llm_with_tools = llm.bind_tools([pdf_scholarship_search, web_search])

In [ ]:
# ✅ Interactive loop
print("\n🎓 Welcome to the Scholarship Assistant!")
print("Type your question below (or type 'exit' to quit):")

messages = []

while True:
    user_input = input("\n🧠 Your question: ").strip()

    if user_input.lower() in ["exit", "quit"]:
        print("👋 Exiting. Good luck with your scholarships!")
        break

    # 1. Add user input
    messages.append(HumanMessage(content=user_input))

    # 2. LLM decides what to do
    ai_message = llm_with_tools.invoke(messages)
    messages.append(ai_message)

    # 3. If tool call, run tool and append result
    if ai_message.tool_calls:
        for call in ai_message.tool_calls:
            tool_name = call["name"]
            tool_args = call["args"]

            if tool_name == "web_search":
                tool_output = web_search.invoke(tool_args)
            elif tool_name == "pdf_scholarship_search":
                tool_output = pdf_scholarship_search.invoke(tool_args)
            else:
                tool_output = f"Unknown tool: {tool_name}"

            # 4. Append tool result as a message
            messages.append(ToolMessage(tool_call_id=call["id"], content=tool_output))

        # 5. Ask LLM to give final answer using tool result
        final_response = llm_with_tools.invoke(messages)
        print("\n📌 Answer:")
        print(final_response.content)
        messages.append(final_response)

    else:
        # No tool needed — just print answer
        print("\n📌 Answer:")
        print(ai_message.content)


🎓 Welcome to the Scholarship Assistant!
Type your question below (or type 'exit' to quit):

🧠 Your question: Who is this scholarship for?

📌 Answer:
This scholarship is for students in India who are pursuing undergraduate degrees. They must have an annual family income below ₹6,00,000 and a minimum of 60% marks in their last qualifying exam.

🧠 Your question: What are the benefits of the scholarship?

📌 Answer:
The scholarship offers ₹10,000 per semester for tuition and a book allowance of ₹3,000 per year.

🧠 Your question: exit
👋 Exiting. Good luck with your scholarships!
